In [8]:
import xarray as xa

import numpy as np
import os
from scipy import interpolate
import pandas as pd

import matplotlib as mpl
from netCDF4 import num2date, date2num
import datetime

import seawater as sw

In [15]:
filein = '/Users/bell/ecoraid/2018/CTDcasts/HE1801/working/he1801_CTDpNutrients.xlsx'

In [16]:
fid = pd.read_excel(filein,skiprows=32)

In [11]:
fid['ST_70']=sw.dens0(s=fid.S_41,t=fid.T_28)-1000.
fid['ST_2070']=sw.dens0(s=fid.S_42,t=fid.T2_35)-1000.

In [14]:
fid.to_csv('/Users/bell/ecoraid/2018/CTDcasts/HE1801/working/he1801_CTDpNutrients.csv')

In [17]:
def O2PercentSat(oxygen_conc=None, temperature=None, salinity=None, pressure=0):
    """
    # calculate oxygen saturation
    # Garcia and Gorden 1992 - from Seabird Derived Parameter Formulas
    
    """

    coefs = {
        "GG_A0": 2.00907,
        "GG_A1": 3.22014,
        "GG_A2": 4.0501,
        "GG_A3": 4.94457,
        "GG_A4": -0.256847,
        "GG_A5": 3.88767,
        "GG_B0": -0.00624523,
        "GG_B1": -0.00737614,
        "GG_B2": -0.010341,
        "GG_B3": -0.00817083,
        "GG_C0": -0.000000488682,
    }

    scaled_temp = np.log((298.15 - temperature) / (273.15 + temperature))

    Oxsol_pri = np.exp(
        coefs["GG_A0"]
        + coefs["GG_A1"] * scaled_temp
        + coefs["GG_A2"] * (scaled_temp) ** 2
        + coefs["GG_A3"] * (scaled_temp) ** 3
        + coefs["GG_A4"] * (scaled_temp) ** 4
        + coefs["GG_A5"] * (scaled_temp) ** 5
        + salinity
        * (
            coefs["GG_B0"]
            + coefs["GG_B1"] * scaled_temp
            + coefs["GG_B2"] * (scaled_temp) ** 2
            + coefs["GG_B3"] * (scaled_temp) ** 3
        )
        + coefs["GG_C0"] * (salinity) ** 2
    )

    # determine sigmatheta and convert Oxygen from micromoles/kg to ml/l
    # calculate new oxygen saturation percent using derived oxsol
    sigmatheta_pri = sw.eos80.pden(salinity, temperature, pressure)
    OxPerSat_pri = ((oxygen_conc * sigmatheta_pri / 44660) / Oxsol_pri) * 100.0

    # replace nan/1e35 with 1e35, >1e10
    try:
        OxPerSat_pri[oxygen_conc >= 1e30] = np.nan
    except:
        if OxPerSat_pri >= 1e30:
            OxPerSat_pri = np.nan

    return OxPerSat_pri


In [20]:
fid['OST_62']=O2PercentSat(oxygen_conc=fid.O_65, temperature=fid.T_28, salinity=fid.S_41, pressure=fid.P_1)
fid['CTDOST_4221']=O2PercentSat(oxygen_conc=fid.CTDOXY_4220, temperature=fid.T2_35, salinity=fid.S_42, pressure=fid.P_1)

In [21]:
fid.to_csv('/Users/bell/ecoraid/2018/CTDcasts/HE1801/working/he1801_CTDpNutrients.csv')